## Waveform Processing -- DSP Optimization Helper

To find an ideal set of DSP parameters, it's helpful to load raw/dsp/hit files and look at a few waveforms.  The `WaveformBrowser` object uses `ProcessingChain` under the hood to process a few waveforms at a time and show you the result.

In [ ]:
import os
import json
import h5py
import argparse
import pandas as pd
import numpy as np
import tinydb as db
from tinydb.storages import MemoryStorage
import matplotlib.pyplot as plt
# plt.style.use('../clint.mpl')
from matplotlib.colors import LogNorm
import boost_histogram as bh
import pickle as pl
from pygama import DataGroup
import pygama.lh5 as lh5
import pygama.analysis.histograms as pgh
import pygama.analysis.peak_fitting as pgf
from pygama.dsp.WaveformBrowser import WaveformBrowser

# do this to use cage_utils functions
import sys
sys.path.insert(1, '../analysis/')
import cage_utils

In [ ]:
run = 244

dsp_id = '06'

user = False
hit = True
cal = True
lowE = False
cut = False

etype = 'trapEftp_cal'

# dsp list for uncalibrated data
# dsp_list = ['energy', 'trapEftp', 'trapEmax', 'bl','bl_sig', 'bl_slope', 'AoE', 'dcr', "tp_0", "tp_02", "tp_05", "tp_10", "tp_20", "tp_30", "tp_40", "tp_50", "tp_60", "tp_70", "tp_80", "tp_90", "tp_96", 'tp_max', 'ToE', 'log_tail_fit_slope', 'wf_max', 'wf_argmax', 'trapE_argmax', 'lf_max']

# dsp list for calibrated data
# dsp_list = ['energy', 'trapEftp', 'trapEmax', 'trapEftp_cal', 'bl','bl_sig', 'bl_slope', 'AoE', 'dcr', "tp_0", "tp_02", "tp_05", "tp_10", "tp_20", "tp_30", "tp_40", "tp_50", "tp_60", "tp_70", "tp_80", "tp_90", "tp_96", 'tp_max', 'ToE', 'log_tail_fit_slope', 'wf_max', 'wf_argmax', 'trapE_argmax', 'lf_max']
dsp_list = ['trapEftp', 'trapEftp_cal', 'bl', 'bl_sig', 'bl_slope']

df_raw, dg, runtype, rt_min, radius, angle_det, rotary = cage_utils.getDataFrame(run, user=user, hit=hit, cal=cal, dsp_list=dsp_list, lowE=lowE)

print(df_raw)

In [ ]:
# get raw files for wfs
raw_lh5_dir = dg.lh5_dir
raw_list = raw_lh5_dir + dg.fileDB['raw_path'] + '/' + dg.fileDB['raw_file']
print(raw_list)

Apply data-cleaning cuts if relevant

In [ ]:
if cut == True:
    print(f'Making cut')
    df_cut = cage_utils.apply_DC_Cuts(run, df_raw)
else:
    df_cut = df_raw
    
print(df_cut)

Make baseline cut if didn't apply other cuts

In [ ]:
# baseline cut
if cut:
    if run <79:
        bl_cut_lo, bl_cut_hi = 9150,9320
    if run>79 and run <117:
        bl_cut_lo, bl_cut_hi = 8500, 10000
    if run>=117:
        bl_cut_lo, bl_cut_hi = 9700, 9760

    df_cut = df.query(f'bl > {bl_cut_lo} and bl < {bl_cut_hi}').copy()
    
    # baseline cut
    bl = df['bl']
    bl_cut = (bl > bl_cut_lo) & (bl < bl_cut_hi)

    # print(cut)
    print(bl_cut.value_counts())

In [ ]:
# energy cut 
energy = df_cut[etype]
# print(df_cut)

if cal == True:
    energy = df_cut['trapEftp_cal']
    energy_cut = (energy > 1450) & (energy < 1470) # 1460 peak
    # energy_cut = (energy > 60) & (energy < 65) # 60 keV peak
    
else:
    # energy = df_hit['trapEmax']
    energy_cut = (energy > 2805) & (energy < 2835) # 1460 peak, uncalibratedm trapEmax
    # energy_cut = (energy > 2000000) & (energy < 2500000) #where alphas seem to be in run 117, raw energy parameter
    # energy_cut = (energy > 200) & (energy < 1000) #where low e weirdness is in DCR, in trapEftp
    
# print(type(energy))
# print(energy_cut)
print(energy_cut.value_counts())

# energy_raw = df['energy']
# energy_cut_raw = (energy_raw > 10200) & (energy_raw < 10600)

# -- set the cut to be used when viewing waveforms -- 
# cut = bl_cut & energy_cut
cut = energy_cut
# print(cut)
print(cut.value_counts())



## DCR / Pole-Zero Correction Plots

Show the raw waveform and the pole-zero corrected waveform used to calculate the DCR (tail slope) parameters.


In [ ]:
dsp_config_file = os.path.expandvars(f'$CAGE_SW/processing/metadata/dsp/dsp_{dsp_id}.json')
print(dsp_id)
browser = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file, # Need to include a dsp config file!
                          # database={"pz_const":'396.9*us'}, # TODO: use metadata instead of manually defining...
                          waveforms=['wf_blsub', 'wf_pzDCR', 'dcr_trap'], # names of waveforms from dsp config file
                          selection = cut, # Apply cut
                          wf_styles=[{'linestyle':['-']},{'linestyle':[':']},{'ls':['--']}],
                          legend=['Waveform', 'PZ Corrected', "DCR: {dcr:0.2f}"],
                          legend_opts={'loc':"lower right"},
                          lines=['dcr'], # add hlines and vlines
                          x_lim=(38000, 80000) # x axis range
                          )
# plt.rcParams['figure.figsize'] = [14, 10]

In [ ]:
%matplotlib widget
browser.draw_next()
# plt.ylim(3100, 3300)
plt.axvline(42500, c='r')
plt.axvline(55000, c='r')

## Energy Plots

In [ ]:
dsp_config_file = os.path.expandvars(f'$CAGE_SW/processing/metadata/dsp/dsp_{dsp_id}.json')
browser2 = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file, 
                           waveforms =['wf_blsub', 'wf_pz', 'wf_trap'],
                           selection = cut, 
                           wf_styles = [{'linestyle':['-']},{'linestyle':[':']},{'ls':['--']}],
                           legend = ['Waveform', 'PZ Corrected',  'trapEftp = {trapEftp:0.2f}'],
                           legend_opts = {'loc':"lower left"},
                           lines = ['trapEftp'],
                           x_lim = (0, 80000)
                           )

In [ ]:
%matplotlib widget
browser2.draw_next()
# plt.ylim(4000, 4500)

## Timepoint calculators -- `tp_0`, etc.

In [ ]:
dsp_config_file = os.path.expandvars(f'$CAGE_SW/processing/metadata/dsp/dsp_{dsp_id}.json')
browser3 = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file, 
                           waveforms = ['wf_blsub', 'wf_pz', 'wf_atrap'],
                           selection = cut,
                           wf_styles = [{'linestyle':['-']},{'linestyle':[':']},{'ls':['--']}],
                           legend = ['Waveform', 'PZ Corrected',  't0 = {tp_0:0.2f}'],
                           lines=['tp_0'],
                           x_lim=(39000, 41000)
                           )

In [ ]:
%matplotlib widget
browser3.draw_next()
# plt.ylim(-10, 200) 
# plt.xlim(38000, 45000)

## T/E calculator

In [ ]:
dsp_config_file = os.path.expandvars(f'$CAGE_SW/processing/metadata/dsp/dsp_{dsp_id}.json')
browser_tri = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file,
                              waveforms = ['wf_blsub', 'wf_pz', 'wf_triangle'],
                              selection = cut,
                              wf_styles=[{'linestyle':['-']},{'linestyle':[':']},{'ls':['--']}],
                              legend=['Waveform', 'PZ Corrected',  'triE= {triE:0.2f}'],
                              lines=['triE'],
                              x_lim=(0, 50000)
                              )

In [ ]:
%matplotlib widget
browser_tri.draw_next()
# plt.ylim(-10, 100) 
plt.xlim(38000, 45000)

## WF_psd
(todo)

## LF noise
Low-frequency maximum value `lf_max`

In [ ]:
dsp_config_file = os.path.expandvars(f'$CAGE_SW/processing/metadata/dsp/dsp_{dsp_id}.json')
browser4 = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file,
                           waveforms = ['wf_psd', 'wf_psd'],
                           selection = cut,
                           wf_styles = [{'linestyle':['-']}],
                           legend = ['lf_max = {lf_max:0.2f}', 'wf_psd'],
                           lines=['lf_max'],
                           x_unit = 'khz',
                           x_lim=(0, 200),
                           verbosity=2
                           )

In [ ]:
%matplotlib widget
browser4.draw_next()
# plt.ylim(0, 2000)

## HF PSD

High-frequency maximum value `hf_max`

In [ ]:
dsp_config_file = os.path.expandvars(f'$CAGE_SW/processing/metadata/dsp/dsp_{dsp_id}.json')
browser5 = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file,
                           waveforms = ['wf_psd', 'wf_psd'],
                           selection = cut, 
                           wf_styles = [{'linestyle':['-']}],
                           legend=['hf_max = {hf_max:0.2f}', 'wf_psd'],
                           lines=['hf_max'],
                           x_unit = 'mhz',
                           x_lim=(0, 50),
                           verbosity=2
                           )

In [ ]:
%matplotlib widget
browser5.draw_next()
# plt.ylim(0, 2000)

## Baseline Slope

In [ ]:
dsp_config_file = os.path.expandvars(f'$CAGE_SW/processing/metadata/dsp/dsp_{dsp_id}.json')
browser6 = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file,
                           waveforms=['wf_blsub', 'wf_pzDCR', 'bl_trap'],
                           selection = cut,
                           wf_styles=[{'linestyle':['-']},{'linestyle':[':']},{'ls':['--']}],
                           legend=['Waveform', 'PZ Corrected', "bl_slope: {bl_slope:0.2f}"],
                           lines=['bl_slope'],
                           x_lim=(0, 20000),
                           )

In [ ]:
%matplotlib widget
browser6.draw_next()
plt.ylim(-30, 30)

In [ ]:
dsp_config_file = os.path.expandvars(f'$CAGE_SW/processing/metadata/dsp/dsp_{dsp_id}.json')
browser7 = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file,
                           waveforms = ['log_tail'],
                           selection = cut,
                           wf_styles=[{'linestyle':['-']}],
                           legend=['log tail'],
                           # x_lim=(0, 80000)
                           )

In [ ]:
%matplotlib widget
browser7.draw_next()
# plt.xlim(0, 40000)